# Linear Regression & GLMs — Student Lab

Complete all TODOs. Avoid sklearn for core parts.

In [ ]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic Dataset (with collinearity)
We generate data where features can be highly correlated to motivate ridge.

In [ ]:
def make_regression(n=400, d=5, noise=0.5, collinear=True):
    X = rng.standard_normal((n, d))
    if collinear and d >= 2:
        X[:, 1] = X[:, 0] * 0.95 + 0.05 * rng.standard_normal(n)
    w_true = rng.standard_normal(d)
    y = X @ w_true + noise * rng.standard_normal(n)
    return X, y, w_true

X, y, w_true = make_regression()
n, d = X.shape
check('shapes', y.shape == (n,))
print('corr(x0,x1)=', np.corrcoef(X[:,0], X[:,1])[0,1])

## Section 1 — OLS Closed Form

### Task 1.1: Closed-form w_hat using solve

# TODO: compute w_hat using solve on (X^T X) w = X^T y
# HINT: `XtX = X.T@X`, `Xty = X.T@y`, `np.linalg.solve(XtX, Xty)`

**Checkpoint:** Why is explicit inverse discouraged?

In [ ]:
# TODO
XtX = ...
Xty = ...
w_hat = ...

check('w_shape', w_hat.shape == (d,))

### Task 1.2: Evaluate fit + residuals
Compute:
- predictions y_pred
- MSE
- residual mean and std

**Interview Angle:** What does a structured residual pattern imply (e.g., nonlinearity)?

In [ ]:
# TODO
y_pred = ...
mse = ...
resid = ...
print('mse', mse, 'resid_mean', resid.mean(), 'resid_std', resid.std())
check('finite', np.isfinite(mse))

## Section 2 — Gradient Descent

### Task 2.1: Implement MSE loss + gradient

Loss = mean((Xw-y)^2), grad = (2/n) X^T(Xw-y)

# TODO: implement `mse_loss_and_grad`

**FAANG gotcha:** shapes and constants.

In [ ]:
def mse_loss_and_grad(X, y, w):
    # TODO
    ...

w0 = np.zeros(d)
loss0, g0 = mse_loss_and_grad(X, y, w0)
check('grad_shape', g0.shape == (d,))
check('finite_loss', np.isfinite(loss0))

### Task 2.2: Train with GD + compare to closed-form

# TODO: implement a simple GD loop, track loss, and compare final weights to w_hat.

**Checkpoint:** How does feature scaling affect GD?

In [ ]:
def train_gd(X, y, lr=0.05, steps=500):
    # TODO
    ...

w_gd, losses = train_gd(X, y, lr=0.05, steps=500)
print('final_loss', losses[-1])
print('||w_gd-w_hat||', np.linalg.norm(w_gd - w_hat))
check('loss_decreases', losses[-1] <= losses[0])

## Section 3 — Ridge Regression (L2)

### Task 3.1: Ridge closed-form
w = (X^T X + λI)^{-1} X^T y

# TODO: implement ridge_solve

**Interview Angle:** Why does ridge help under collinearity?

In [ ]:
def ridge_solve(X, y, lam):
    # TODO
    ...

w_ridge = ridge_solve(X, y, lam=1.0)
check('ridge_shape', w_ridge.shape == (d,))

### Task 3.2: Bias/variance demo with train/test split

# TODO: split into train/test and compare MSE for multiple lambdas.

**Checkpoint:** why can test error improve even when train error worsens?

In [ ]:
# TODO
idx = rng.permutation(n)
train = idx[: int(0.7*n)]
test = idx[int(0.7*n):]
Xtr, ytr = X[train], y[train]
Xte, yte = X[test], y[test]

lams = [0.0, 0.1, 1.0, 10.0]
results = []
for lam in lams:
    w = ridge_solve(Xtr, ytr, lam=lam) if lam > 0 else np.linalg.solve(Xtr.T@Xtr, Xtr.T@ytr)
    tr_mse = np.mean((Xtr@w - ytr)**2)
    te_mse = np.mean((Xte@w - yte)**2)
    results.append((lam, tr_mse, te_mse))
print('lam, train_mse, test_mse')
for r in results:
    print(r)

## Section 4 — GLM Intuition

### Task 4.1: Match tasks to (distribution, link)
Fill in a table for:
- regression
- binary classification
- count prediction

**Explain:** what changes when you go from OLS to a GLM?

| Problem | Target type | Distribution | Link | Loss |
|---|---|---|---|---|
| House price | continuous | ? | ? | ? |
| Fraud | binary | ? | ? | ? |
| Clicks per user | count | ? | ? | ? |


---
## Submission Checklist
- All TODOs completed
- Train/test results shown for ridge
- Short answers to checkpoint questions
